In [18]:
!git lfs install

Git LFS initialized.


In [1]:
pip install SpeechRecognition pydub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 20.6 MB/s eta 0:00:00


In [12]:
import speech_recognition as sr
import os
import subprocess
from google.colab import files
from pydub import AudioSegment
from pydub.silence import split_on_silence

In [13]:
files.upload()

Saving 01.mp3 to 01.mp3


{'01.mp3': b'\xff\xf3\x84\xc4\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xf3\x84\xc4\x00\'\x94\x12\x08\x00\xc2F\xdd\x03\x00\xa83\x8bS\x8e0\xec9\x0f\xc7;7\x0f\xc6\xe3s\xf5#\xee<\xa1\xc8\x8aT\x0cFB\x87 \x82d\xfa\xba=\xaa@@\x82\x082\xa1$\x10PP\xc1\x01&y\xd3\x17\xe9\x03\x13#F\xdf\\\x9d\xba\

In [14]:
# convert mp3 to wav file
subprocess.call(['ffmpeg', '-i', '01.mp3','wav_file.wav'])

0

In [15]:
# initialize the recognizer
r = sr.Recognizer()

# a function that splits the audio file into chunks
# and applies speech recognition

def get_large_audio_transcription(path):
    """
    Splitting the large audio file into chunks
    and apply speech recognition on each of these chunks
    """
    # open the audio file using pydub
    sound = AudioSegment.from_wav(path)
    # split audio sound where silence is 700 miliseconds or more and get chunks
    chunks = split_on_silence(sound,
        # experiment with this value for your target audio file
        min_silence_len = 500,
        # adjust this per requirement
        silence_thresh = sound.dBFS-14,
        # keep the silence for 1 second, adjustable as well
        keep_silence=500,
    )
    folder_name = "audio-chunks"
    # create a directory to store the audio chunks
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
    whole_text = ""
    # process each chunk
    for i, audio_chunk in enumerate(chunks, start=1):
        # export audio chunk and save it in
        # the `folder_name` directory.
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")
        # recognize the chunk
        with sr.AudioFile(chunk_filename) as source:
            audio_listened = r.record(source)
            # try converting it to text
            try:
                text = r.recognize_google(audio_listened)
            except sr.UnknownValueError as e:
                print("Error:", str(e))
            else:
                text = f"{text.capitalize()}. "
                print(chunk_filename, ":", text)
                whole_text += text
    # return the text for all chunks detected
    return whole_text

In [16]:
path = "/content/wav_file.wav"
print("\nFull text:", get_large_audio_transcription(path))

audio-chunks/chunk1.wav : Hello guys. 
audio-chunks/chunk2.wav : I'm an engineering student. 
audio-chunks/chunk3.wav : I'm doing my btech from baba banda singh bahadur engineering college. 
Error: 
audio-chunks/chunk5.wav : Hobbies are traveling world. 
audio-chunks/chunk6.wav : Play badminton reading books. 
audio-chunks/chunk7.wav : Etcetera. 

Full text: Hello guys. I'm an engineering student. I'm doing my btech from baba banda singh bahadur engineering college. Hobbies are traveling world. Play badminton reading books. Etcetera. 
